## Random

### Configuración Inicial

In [1]:
!pip uninstall -y numpy
!pip install numpy==1.26

Found existing installation: numpy 1.26.0
Uninstalling numpy-1.26.0:
  Successfully uninstalled numpy-1.26.0
  Using cached numpy-1.26.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (58 kB)
Using cached numpy-1.26.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.9 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.0 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.0 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.0 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.0 which is incompatible.


In [2]:
!pip install scikit-surprise --no-build-isolation --no-deps
!pip install memory_profiler

  Using cached scikit_surprise-1.1.4.tar.gz (154 kB)
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp312-cp312-linux_x86_64.whl size=2708529 sha256=d07f1241379f07f87782d65059757531a91278ec43adc54b5e2b9fb66e4eb1f2
  Stored in directory: /root/.cache/pip/wheels/75/fa/bc/739bc2cb1fbaab6061854e6cfbb81a0ae52c92a502a7fa454b
Successfully built scikit-surprise


### Instalación de Librerías

In [20]:
import time
import json
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict, Counter
from memory_profiler import memory_usage
import itertools
import scipy.sparse as sparse
import random
import gdown
from surprise import SVDpp, Dataset, Reader, accuracy
from sklearn.metrics import mean_squared_error, mean_absolute_error

### Importación de los Datos

In [4]:
gdown.download(id='1H_24ycns6zbOVfHFJRI9vGjVffVA5z6v', output='training_ratings.csv', quiet=False)
gdown.download(id='1pKmf07ehHOmlvIyT8nv__vPuWE2Z3ygZ', output='validation_ratings.csv', quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1H_24ycns6zbOVfHFJRI9vGjVffVA5z6v
From (redirected): https://drive.google.com/uc?id=1H_24ycns6zbOVfHFJRI9vGjVffVA5z6v&confirm=t&uuid=03082100-ddd2-4a7a-bfed-69390ceb203d
To: /content/training_ratings.csv
100%|██████████| 249M/249M [00:03<00:00, 67.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1pKmf07ehHOmlvIyT8nv__vPuWE2Z3ygZ
To: /content/validation_ratings.csv
100%|██████████| 58.3M/58.3M [00:01<00:00, 57.8MB/s]


'validation_ratings.csv'

In [5]:
df_train = pd.read_csv('training_ratings.csv')
df_val = pd.read_csv('validation_ratings.csv')

### Preprocesamiento de Datos

In [6]:
df_train.drop_duplicates(inplace=True, subset=['user', 'item'])
df_val.drop_duplicates(inplace=True, subset=['user', 'item'])

In [17]:
print(f"Tamaño original del training set: {len(df_train)}")

# se obtiene un sample debido a que hay muchos datos y se demora mucho
df_train_sample = df_train.sample(n=1000000, random_state=42)
print(f"Tamaño del nuevo training set (muestra): {len(df_train_sample)}")

# se obtiene un sample debido a que hay muchos datos y se demora mucho
df_val_sample = df_val.sample(n=50000, random_state=42)
print(f"Tamaño del nuevo validation set (muestra): {len(df_val_sample)}")

Tamaño original del training set: 12390406
Tamaño del nuevo training set (muestra): 1000000
Tamaño del nuevo validation set (muestra): 50000


In [18]:
def evaluar_random_topn(df_train, df_val, n=10, sample_per_user=50):
    """
    Genera recomendaciones aleatorias para cada usuario, tomando un sample
    limitado de items no vistos para evitar usar toda la matriz.
    """
    # Diccionario {usuario: items que ha visto}
    user2seen = df_train.groupby('user')['item'].apply(set).to_dict()

    # Lista de todos los items
    all_items = df_train['item'].unique().tolist()

    top_n = {}
    for uid in df_val['user'].unique():
        seen = user2seen.get(uid, set())
        # Items posibles para recomendar
        candidates = list(set(all_items) - seen)
        # Tomar un sample limitado
        sample_candidates = random.sample(candidates, min(sample_per_user, len(candidates)))
        # Tomar n recomendaciones aleatorias
        recs = random.sample(sample_candidates, min(n, len(sample_candidates)))
        top_n[uid] = [(iid, random.randint(1,5)) for iid in recs]

    return top_n


# Medir tiempo de ejecución
start_time = time.time()
top_n = evaluar_random_topn(df_train_sample, df_val_sample, n=10)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Tiempo de ejecución: {elapsed_time:.2f} segundos")

# Medir la memoria utilizada
memoria = memory_usage(
    (evaluar_random_topn, (df_val_sample, df_val_sample), {'n':10})
)
print("Memoria usada (MB):", max(memoria) - min(memoria))

Tiempo de ejecución: 119.73 segundos
Memoria usada (MB): 21.0


In [23]:
def rmse_mae_from_topn(top_n, df_val_sample):
    real, predicho = [], []
    total = sum(len(recs) for recs in top_n.values())
    i = 0
    for uid, recs in top_n.items():
      for iid, pred in recs:
        real_vals = df_val_sample.loc[(df_val_sample['user'] == uid) & (df_val_sample['item'] == iid), 'rating']
        if not real_vals.empty:
            real.append(real_vals.values[0])
            predicho.append(pred)
        i += 1
        if i % 10000 == 0 or i == total:  # muestra cada 100 pasos o al final
            progreso = (i / total) * 100
            print(f"Progreso: {i}/{total} ({progreso:.2f}%)")

    return math.sqrt(mean_squared_error(real, predicho)), mean_absolute_error(real, predicho)

rmse, mae = rmse_mae_from_topn(top_n, df_val_sample)
print("RMSE para las top n recomendaciones", rmse)
print("MAE para las top n recomendaciones", mae)


Progreso: 10000/374620 (2.67%)
Progreso: 20000/374620 (5.34%)
Progreso: 30000/374620 (8.01%)
Progreso: 40000/374620 (10.68%)
Progreso: 50000/374620 (13.35%)
Progreso: 60000/374620 (16.02%)
Progreso: 70000/374620 (18.69%)
Progreso: 80000/374620 (21.35%)
Progreso: 90000/374620 (24.02%)
Progreso: 100000/374620 (26.69%)
Progreso: 110000/374620 (29.36%)
Progreso: 120000/374620 (32.03%)
Progreso: 130000/374620 (34.70%)
Progreso: 140000/374620 (37.37%)
Progreso: 150000/374620 (40.04%)
Progreso: 160000/374620 (42.71%)
Progreso: 170000/374620 (45.38%)
Progreso: 180000/374620 (48.05%)
Progreso: 190000/374620 (50.72%)
Progreso: 200000/374620 (53.39%)
Progreso: 210000/374620 (56.06%)
Progreso: 220000/374620 (58.73%)
Progreso: 230000/374620 (61.40%)
Progreso: 240000/374620 (64.06%)
Progreso: 250000/374620 (66.73%)
Progreso: 260000/374620 (69.40%)
Progreso: 270000/374620 (72.07%)
Progreso: 280000/374620 (74.74%)
Progreso: 290000/374620 (77.41%)
Progreso: 300000/374620 (80.08%)
Progreso: 310000/37462

Ahora haremos lo mismo pero para un grupo de 4 personas que han calificado ese item (haremos 4 personas por mas que haya juegos que es de máximo 2 o 10 etc por simplicidad). Lo que hacemos es calcular el promedio real de las calificaciones de esas 4 personas y compararlo contra una predicción grupal generada de manera completamente aleatoria en el rango de ratings posibles.

In [24]:
def evaluar_random_topn_grupos(df_train, df_val, n=10, sample_per_user=50):
    """
    Genera recomendaciones aleatorias para cada usuario, tomando un sample
    limitado de items no vistos para evitar usar toda la matriz.
    """
    # Diccionario {usuario: items que ha visto}
    user2seen = df_train.groupby('user')['item'].apply(set).to_dict()

    # Lista de todos los items
    all_items = df_train['item'].unique().tolist()

    top_n = {}
    # armar grupos
    usuarios = df_val['user'].unique()
    grupos = [usuarios[i:i+4] for i in range(0, len(usuarios) - len(usuarios)%4, 4)]

    seen_group = set()
    for grupo in grupos:
      for u in grupo:
          seen_group |= user2seen.get(u, set())
          # Items posibles para recomendar
          candidates = list(set(all_items) - seen_group)
          # Tomar un sample limitado
          sample_candidates = random.sample(candidates, min(sample_per_user, len(candidates)))
          # Tomar n recomendaciones aleatorias
          recs = random.sample(sample_candidates, min(n, len(sample_candidates)))
          top_n[tuple(grupo)] = [(iid, random.randint(1,5)) for iid in recs]

    return top_n


# Medir tiempo de ejecución
start_time = time.time()
top_n_grupo = evaluar_random_topn_grupos(df_train_sample, df_val_sample, n=10)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Tiempo de ejecución: {elapsed_time:.2f} segundos")

# Medir la memoria utilizada
memoria = memory_usage(
    (evaluar_random_topn_grupos, (df_train_sample, df_val_sample), {'n':10})
)
print("Memoria usada (MB):", max(memoria) - min(memoria))

Tiempo de ejecución: 139.43 segundos
Memoria usada (MB): 82.625


In [26]:
def rmse_mae_from_topn_grupo(top_n_grupo, df_val_sample):
    real, predicho = [], []
    total = sum(len(recs) for recs in top_n.values())
    i = 0
    for grupo, recs in top_n_grupo.items():
      for iid, pred in recs:
        real_vals = df_val_sample.loc[(df_val_sample['user'].isin(grupo)) & (df_val_sample['item'] == iid), 'rating']
        if not real_vals.empty:
            real.append(real_vals.values[0])
            predicho.append(pred)
        i += 1
        if i % 10000 == 0 or i == total:  # muestra cada 100 pasos o al final
            progreso = (i / total) * 100
            print(f"Progreso: {i}/{total} ({progreso:.2f}%)")

    return math.sqrt(mean_squared_error(real, predicho)), mean_absolute_error(real, predicho)

rmse_grupo, mae_grupo = rmse_mae_from_topn_grupo(top_n_grupo, df_val_sample)
print("RMSE para las top n recomendaciones", rmse_grupo)
print("MAE para las top n recomendaciones", mae_grupo)


Progreso: 10000/374620 (2.67%)
Progreso: 20000/374620 (5.34%)
Progreso: 30000/374620 (8.01%)
Progreso: 40000/374620 (10.68%)
Progreso: 50000/374620 (13.35%)
Progreso: 60000/374620 (16.02%)
Progreso: 70000/374620 (18.69%)
Progreso: 80000/374620 (21.35%)
Progreso: 90000/374620 (24.02%)
RMSE para las top n recomendaciones 3.5
MAE para las top n recomendaciones 3.5


Los códigos para random se adaptaron de un codigo inicial creado, la adaptación de este código se encuentra aquí: https://chatgpt.com/share/68e00c31-dbf8-8006-bacd-84f0296d467c
